In [1]:
import proplot as pplt
import numpy as np
from mdet_tools import meas_m, meas_mbmeds, make_obs

%load_ext autoreload
%autoreload 2

In [ ]:
n_stars = 20
m, merr, c, cerr, _, _ = meas_m(n_stars=n_stars, mask_width=4, seed=20, n_jobs=10)

print("\nstar density: %s [num per arcmin^2]" % (n_stars / (235*0.20/60)**2))
mstr = "m +/- merr: %0.6f +/- %0.6f [10^(-3), 3sigma]" % (m/1e-3, 3*merr/1e-3)
print(mstr)

cstr = "c +/- cerr: %0.6f +/- %0.6f [10^(-5), 3sigma]" % (c/1e-3, 3*cerr/1e-3)
print(cstr)

  0%|                                                   | 0/10 [00:00<?, ?it/s]

In [ ]:
step = 0.01
s = 100
kwargs = {"n_stars": n_stars, "seed": s}
d, mbobs = meas_mbmeds(make_obs(mcal_shear=(0, 0), **kwargs))
d1p, mbobs1p = meas_mbmeds(make_obs(mcal_shear=(step, 0), **kwargs))
d1m, mbobs1m = meas_mbmeds(make_obs(mcal_shear=(-step, 0), **kwargs))
d = _cut_cat(d)
d1p = _cut_cat(d1p)
d1m = _cut_cat(d1m)

fig, axs = pplt.subplots(nrows=1, ncols=3, sharex=3, sharey=3)
axs[0].pcolormesh(mbobs[0][0].image)
axs[0].plot(d["x"], d["y"], '.b')
axs[0].set_title(mstr)

axs[1].pcolormesh(mbobs1p[0][0].image)
axs[1].plot(d1p["x"], d1p["y"], '.b')
axs[1].set_title("mcal +g1")

axs[2].pcolormesh(mbobs1m[0][0].image)
axs[2].plot(d1m["x"], d1m["y"], '.b')
axs[2].set_title("mcal -g1")

axs.set_xlim(0, mbobs[0][0].image.shape[1])
axs.set_ylim(0, mbobs[0][0].image.shape[0])